<a href="https://colab.research.google.com/github/davidlealo/asistenteabp/blob/main/agenteABP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalar bibliotecas necesarias

In [1]:
! pip install mistralai
!pip install transformers ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.0/260.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 1.9 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.0
    Uninstalling httpx-0.28.0:
      Successfully uninstalled httpx-0.28.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.2 MB/s eta 0:00:00


# Solicitar Clave Mistral

In [2]:
from IPython.display import display, HTML, Javascript
from transformers import AutoModelForCausalLM, AutoTokenizer

# Interfaz para ingresar la API key
def request_api_key():
    input_html = """
    <div style="font-family: Arial; margin: 10px; padding: 10px; border: 1px solid #ddd; border-radius: 5px;">
        <h3>Proveer API Key</h3>
        <p>Por favor, ingrese su API key de Mistral en el siguiente campo:</p>
        <input type="password" id="apiKeyInput" placeholder="Ingrese su API key aquí" style="width: 100%; padding: 8px; margin-bottom: 10px;">
        <button onclick="storeApiKey()" style="padding: 10px 15px; background-color: #007BFF; color: white; border: none; border-radius: 3px; cursor: pointer;">Guardar API Key</button>
    </div>
    <script>
        function storeApiKey() {
            const apiKey = document.getElementById('apiKeyInput').value;
            if (apiKey) {
                google.colab.kernel.invokeFunction('notebook.store_api_key', [apiKey], {});
                alert('API key almacenada correctamente.');
            } else {
                alert('Por favor, ingrese una API key válida.');
            }
        }
    </script>
    """
    display(HTML(input_html))

# Callback para almacenar la API key en el entorno Python
def store_api_key(api_key):
    global mistral_api_key
    mistral_api_key = api_key
    print("API key almacenada correctamente.")

# Registrar la función en Google Colab
from google.colab import output
output.register_callback('notebook.store_api_key', store_api_key)

# Solicitar la API key al usuario
request_api_key()


API key almacenada correctamente.


In [5]:
import requests

# Define la función para interactuar con la API de Mistral
def run_mistral(user_message, model="mistral-large-latest", api_key=None):
    if not api_key:
        raise ValueError("Por favor, proporciona una API key válida para Mistral.")

    # URL base de la API (esto puede variar dependiendo de la documentación de Mistral)
    api_url = f"https://api.mistral.ai/v1/chat/completions"

    # Configura el payload para la solicitud
    payload = {
        "model": model,
        "messages": [
            {"role": "user", "content": user_message}
        ]
    }

    # Cabeceras de autenticación
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    # Envía la solicitud POST
    response = requests.post(api_url, json=payload, headers=headers)

    # Manejo de errores
    if response.status_code != 200:
        raise Exception(f"Error en la API de Mistral: {response.status_code} {response.text}")

    # Devuelve la respuesta generada
    response_data = response.json()
    return response_data["choices"][0]["message"]["content"]



In [6]:
from IPython.display import display, HTML

# Ejemplo de proyecto generado por el agente
project_example = """
Título del Proyecto: ¿Cómo respiran los huevos?
Descripción: Este proyecto permite a los estudiantes investigar cómo los pollitos respiran dentro del huevo, utilizando experimentos sobre la porosidad de la cáscara.
Habilidades del siglo XXI: Pensamiento Crítico, Colaboración y Comunicación.
Fases:
1. Planteamiento de la pregunta guía.
2. Investigación experimental utilizando colorantes.
3. Creación de una representación artística.
4. Presentación de resultados y reflexiones.
"""

# Generar evaluación interactiva
def display_project_evaluation(project):
    evaluation_html = f"""
    <div style="font-family: Arial; margin: 10px; padding: 10px; border: 1px solid #ddd; border-radius: 5px;">
        <h3>Proyecto Generado</h3>
        <p>{project}</p>
        <h4>Evaluación del Proyecto</h4>
        <form id="evaluationForm">
            <label for="relevance">Relevancia del proyecto:</label><br>
            <input type="radio" id="relevance1" name="relevance" value="1"> 1 - Baja<br>
            <input type="radio" id="relevance2" name="relevance" value="2"> 2<br>
            <input type="radio" id="relevance3" name="relevance" value="3"> 3<br>
            <input type="radio" id="relevance4" name="relevance" value="4"> 4<br>
            <input type="radio" id="relevance5" name="relevance" value="5"> 5 - Alta<br><br>

            <label for="clarity">Claridad de las instrucciones:</label><br>
            <input type="radio" id="clarity1" name="clarity" value="1"> 1 - Baja<br>
            <input type="radio" id="clarity2" name="clarity" value="2"> 2<br>
            <input type="radio" id="clarity3" name="clarity" value="3"> 3<br>
            <input type="radio" id="clarity4" name="clarity" value="4"> 4<br>
            <input type="radio" id="clarity5" name="clarity" value="5"> 5 - Alta<br><br>

            <label for="comments">Comentarios adicionales:</label><br>
            <textarea id="comments" name="comments" rows="4" cols="50"></textarea><br><br>

            <button type="button" onclick="submitEvaluation()">Enviar Evaluación</button>
        </form>
    </div>
    <script>
        function submitEvaluation() {{
            const relevance = document.querySelector('input[name="relevance"]:checked')?.value || "Sin seleccionar";
            const clarity = document.querySelector('input[name="clarity"]:checked')?.value || "Sin seleccionar";
            const comments = document.getElementById('comments').value || "Sin comentarios";
            alert(`Evaluación enviada:\\nRelevancia: ${relevance}\\nClaridad: ${clarity}\\nComentarios: ${comments}`);
        }}
    </script>
    """
    display(HTML(evaluation_html))

# Mostrar la evaluación del proyecto
display_project_evaluation(project_example)


NameError: name 'relevance' is not defined